# 예보 데이터 불러오기
[한국동서발전 태양광 발전량 현황 인증키 확인](https://data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15057210)
[인증키 주소(로그인필요..)](https://data.go.kr/iim/api/selectAPIAcountView.do)

In [ ]:
! pip install xmltodict

In [1]:
import pandas as pd
from urllib.parse import urlencode, quote_plus
from urllib.request import urlopen
import json
import xmltodict

In [165]:
# API
ServiceKey = input('encoded api key를 입력해주세요 > ')

encoded api key를 입력해주세요 > ㅁ


In [142]:
def get_obs_ulsan(start, end, key):
    '''
    예시
    start = pd.Timestamp('2018-03-01 00:00:00')
    end = pd.Timestamp('2018-03-31 00:00:00')
    key = ServiceKey
    
    위와 같이 넣으면 2018년 3월 동안
    매 시간 울산의 obs data를 가져온다.
    '''
    startDt = f'{start.year}{start.month:0>2d}{start.day:0>2d}'
    startHh = '00'
    endDt = f'{end.year}{end.month:0>2d}{end.day:0>2d}'
    endHh = '23'
    
    url_base = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'
    
    date_start = pd.to_datetime(start)
    date_end = pd.to_datetime(end)
    num_hours = int((date_end - date_start) / pd.Timedelta(hours=1)) + 1
    
    queryParams = urlencode({
        quote_plus('pageNo'): 1,
        quote_plus('numOfRows'): num_hours,
        quote_plus('dataCd'): 'ASOS',
        quote_plus('dateCd'): 'HR',
        quote_plus('stnIds'): 152,               # ulsan
        quote_plus('startDt'): startDt,
        quote_plus('startHh'): startHh,
        quote_plus('endDt'): endDt,
        quote_plus('endHh'): endHh,
    })
    
    url = f'{url_base}?serviceKey={key}&{queryParams}'
    response = urlopen(url)
    results = response.read().decode('utf-8')
    results_json = xmltodict.parse(results)

    data = json.loads(json.dumps(results_json))

    if 'ERROR' in data['response']['header']['resultMsg']:
        print('Fail.')
    else:
        print('Success!')
        
    logs = data['response']['body']['items']['item']
    df = pd.DataFrame(logs)
    
    col_need_origin = ['tm', 'stnId', 'stnNm', 'ta', 'ws', 'wd', 'hm', 'dc10Tca']
    col_need = [
        'tm', 'stnId', 'stnNm', 'ta', 'ws', 'wd', 'hm', 'dc10Tca',
        'rn', 'pv', 'ps', 'ss', 'dc10LmcsCa',
    ]
    dict_col = {
        'stnId': '지점',
        'stnNm': '지점명',
        'tm': '일시',
        'ta': '기온(°C)',
        'ws': '풍속(m/s)',
        'wd': '풍향(16방위)',
        'hm': '습도(%)',
        'dc10Tca': '전운량(10분위)',
        'rn': '강수량(mm)',        # 추가
        'pv': '증기압(hPa)',
        'ps': '해면기압(hPa)',
        'ss': '일조(hr)',
        'dc10LmcsCa': '중하층운량(10분위)'
    }

    weather = df[col_need].copy()
    weather = weather.rename(columns=dict_col)
    
    return weather


def get_obs_dangjin(start, end, key):
    '''
    예시
    start = pd.Timestamp('2018-03-01 00:00:00')
    end = pd.Timestamp('2018-03-31 00:00:00')
    key = ServiceKey
    
    위와 같이 넣으면 2018년 3월 동안
    매 시간 당진(서산)의 obs data를 가져온다.
    '''
    startDt = f'{start.year}{start.month:0>2d}{start.day:0>2d}'
    startHh = '00'
    endDt = f'{end.year}{end.month:0>2d}{end.day:0>2d}'
    endHh = '23'
    
    url_base = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'
    
    date_start = pd.to_datetime(start)
    date_end = pd.to_datetime(end)
    num_hours = int((date_end - date_start) / pd.Timedelta(hours=1)) + 1
    
    queryParams = urlencode({
        quote_plus('pageNo'): 1,
        quote_plus('numOfRows'): num_hours,
        quote_plus('dataCd'): 'ASOS',
        quote_plus('dateCd'): 'HR',
        quote_plus('stnIds'): 129,               # 서산 (당진 옆)
        quote_plus('startDt'): startDt,
        quote_plus('startHh'): startHh,
        quote_plus('endDt'): endDt,
        quote_plus('endHh'): endHh,
    })
    
    url = f'{url_base}?serviceKey={key}&{queryParams}'
    response = urlopen(url)
    results = response.read().decode('utf-8')
    results_json = xmltodict.parse(results)

    data = json.loads(json.dumps(results_json))

    if 'ERROR' in data['response']['header']['resultMsg']:
        print('Fail.')
    else:
        print('Success!')
        
    logs = data['response']['body']['items']['item']
    df = pd.DataFrame(logs)
    
    col_need_origin = ['tm', 'stnId', 'stnNm', 'ta', 'ws', 'wd', 'hm', 'dc10Tca']
    col_need = [
        'tm', 'stnId', 'stnNm', 'ta', 'ws', 'wd', 'hm', 'dc10Tca',
        'rn', 'pv', 'ps', 'ss', 'dc10LmcsCa',
    ]
    dict_col = {
        'stnId': '지점',
        'stnNm': '지점명',
        'tm': '일시',
        'ta': '기온(°C)',
        'ws': '풍속(m/s)',
        'wd': '풍향(16방위)',
        'hm': '습도(%)',
        'dc10Tca': '전운량(10분위)',
        'rn': '강수량(mm)',        # 추가
        'pv': '증기압(hPa)',
        'ps': '해면기압(hPa)',
        'ss': '일조(hr)',
        'dc10LmcsCa': '중하층운량(10분위)'
    }

    weather = df[col_need].copy()
    weather = weather.rename(columns=dict_col)
    
    return weather

In [104]:
pd.date_range('20150101', '20210529', freq='M')[0]

Timestamp('2015-01-31 00:00:00', freq='M')

In [158]:
# ulsan data 가져오기
obs_ulsan = pd.DataFrame()

date_start = '20150101'
date_end = '20210529'

month_start = pd.date_range(date_start, date_end, freq='MS')
month_end = pd.date_range(date_start, date_end, freq='M')

obs_ulsan = pd.DataFrame()
for month in range(len(month_end)):
    tmp = get_obs_ulsan(month_start[month], month_end[month], ServiceKey)
    obs_ulsan = pd.concat([obs_ulsan, tmp])
    print(f'trial {month} : {month_start[month]} ~ {month_end[month]}')

print('Done!')


Success!
trial 0 : 2015-01-01 00:00:00 ~ 2015-01-31 00:00:00
Success!
trial 1 : 2015-02-01 00:00:00 ~ 2015-02-28 00:00:00
Success!
trial 2 : 2015-03-01 00:00:00 ~ 2015-03-31 00:00:00
Success!
trial 3 : 2015-04-01 00:00:00 ~ 2015-04-30 00:00:00
Success!
trial 4 : 2015-05-01 00:00:00 ~ 2015-05-31 00:00:00
Success!
trial 5 : 2015-06-01 00:00:00 ~ 2015-06-30 00:00:00
Success!
trial 6 : 2015-07-01 00:00:00 ~ 2015-07-31 00:00:00
Success!
trial 7 : 2015-08-01 00:00:00 ~ 2015-08-31 00:00:00
Success!
trial 8 : 2015-09-01 00:00:00 ~ 2015-09-30 00:00:00
Success!
trial 9 : 2015-10-01 00:00:00 ~ 2015-10-31 00:00:00
Success!
trial 10 : 2015-11-01 00:00:00 ~ 2015-11-30 00:00:00
Success!
trial 11 : 2015-12-01 00:00:00 ~ 2015-12-31 00:00:00
Success!
trial 12 : 2016-01-01 00:00:00 ~ 2016-01-31 00:00:00
Success!
trial 13 : 2016-02-01 00:00:00 ~ 2016-02-29 00:00:00
Success!
trial 14 : 2016-03-01 00:00:00 ~ 2016-03-31 00:00:00
Success!
trial 15 : 2016-04-01 00:00:00 ~ 2016-04-30 00:00:00
Success!
trial 16 

In [159]:
obs_ulsan.to_csv('new_obs_ulsan.csv', index = False)

In [160]:
# dangjin data 가져오기
obs_dangjin = pd.DataFrame()

date_start = '20150101'
date_end = '20210529'

month_start = pd.date_range(date_start, date_end, freq='MS')
month_end = pd.date_range(date_start, date_end, freq='M')

obs_dangjin = pd.DataFrame()
for month in range(len(month_end)):
    tmp = get_obs_dangjin(month_start[month], month_end[month], ServiceKey)
    obs_dangjin = pd.concat([obs_dangjin, tmp])
    print(f'trial {month} : {month_start[month]} ~ {month_end[month]}')

print('Done!')


Success!
trial 0 : 2015-01-01 00:00:00 ~ 2015-01-31 00:00:00
Success!
trial 1 : 2015-02-01 00:00:00 ~ 2015-02-28 00:00:00
Success!
trial 2 : 2015-03-01 00:00:00 ~ 2015-03-31 00:00:00
Success!
trial 3 : 2015-04-01 00:00:00 ~ 2015-04-30 00:00:00
Success!
trial 4 : 2015-05-01 00:00:00 ~ 2015-05-31 00:00:00
Success!
trial 5 : 2015-06-01 00:00:00 ~ 2015-06-30 00:00:00
Success!
trial 6 : 2015-07-01 00:00:00 ~ 2015-07-31 00:00:00
Success!
trial 7 : 2015-08-01 00:00:00 ~ 2015-08-31 00:00:00
Success!
trial 8 : 2015-09-01 00:00:00 ~ 2015-09-30 00:00:00
Success!
trial 9 : 2015-10-01 00:00:00 ~ 2015-10-31 00:00:00
Success!
trial 10 : 2015-11-01 00:00:00 ~ 2015-11-30 00:00:00
Success!
trial 11 : 2015-12-01 00:00:00 ~ 2015-12-31 00:00:00
Success!
trial 12 : 2016-01-01 00:00:00 ~ 2016-01-31 00:00:00
Success!
trial 13 : 2016-02-01 00:00:00 ~ 2016-02-29 00:00:00
Success!
trial 14 : 2016-03-01 00:00:00 ~ 2016-03-31 00:00:00
Success!
trial 15 : 2016-04-01 00:00:00 ~ 2016-04-30 00:00:00
Success!
trial 16 

In [162]:
# 저장하기
obs_dangjin.to_csv('new_obs_dangjin.csv', index = False)

In [163]:
obs_dangjin

,일시,지점,지점명,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),강수량(mm),증기압(hPa),해면기압(hPa),일조(hr),중하층운량(10분위)
0,2015-01-01 00:00,129,서산,None,5.3,340,None,None,0.2,None,None,None,None
1,2015-01-01 01:00,129,서산,-4.4,5.4,340,47,6,None,2.1,1024.0,None,6
2,2015-01-01 02:00,129,서산,-4.6,4.9,340,50,None,None,2.2,1024.0,None,None
3,2015-01-01 03:00,129,서산,-4.7,6.2,320,50,6,0.0,2.2,1024.5,None,6
4,2015-01-01 04:00,129,서산,-5.0,5.0,320,56,6,None,2.4,1024.4,None,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,2021-04-29 20:00,129,서산,12.3,0.9,200,90,10,0.2,12.9,1004.2,0.0,None
693,2021-04-29 21:00,129,서산,12.2,1.5,90,93,10,0.3,13.1,1003.5,None,None
694,2021-04-29 22:00,129,서산,11.9,2.5,160,88,10,0.8,12.2,1004.2,None,None
695,2021-04-29 23:00,129,서산,10.7,2.7,110,87,10,2.9,11.2,1001.6,None,None


# 이하 무시해도 되는 코드

In [140]:
start = month_start[month]
end = month_end[month]
key = ServiceKey

startDt = f'{start.year}{start.month:0>2d}{start.day:0>2d}'
startHh = '00'
endDt = f'{end.year}{end.month:0>2d}{end.day:0>2d}'
endHh = '23'

url_base = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'

date_start = pd.to_datetime(start)
date_end = pd.to_datetime(end)
num_hours = int((date_end - date_start) / pd.Timedelta(hours=1)) + 1

queryParams = urlencode({
    quote_plus('pageNo'): 1,
    quote_plus('numOfRows'): num_hours,
    quote_plus('dataCd'): 'ASOS',
    quote_plus('dateCd'): 'HR',
    quote_plus('stnIds'): 152,               # ulsan
    quote_plus('startDt'): startDt,
    quote_plus('startHh'): startHh,
    quote_plus('endDt'): endDt,
    quote_plus('endHh'): endHh,
})

url = f'{url_base}?serviceKey={key}&{queryParams}'
response = urlopen(url)
results = response.read().decode('utf-8')
results_json = xmltodict.parse(results)

data = json.loads(json.dumps(results_json))

if 'ERROR' in data['response']['header']['resultMsg']:
    print('Fail.')
else:
    print('Success!')

Success!
20150101


In [141]:
logs = data['response']['body']['items']['item']
df = pd.DataFrame(logs)

col_need_origin = ['tm', 'stnId', 'stnNm', 'ta', 'ws', 'wd', 'hm', 'dc10Tca']
col_need = [
    'tm', 'stnId', 'stnNm', 'ta', 'ws', 'wd', 'hm', 'dc10Tca',
    'rn', 'pv', 'ps', 'ss', 'dc10LmcsCa',
]
dict_col = {
    'stnId': '지점',
    'stnNm': '지점명',
    'tm': '일시',
    'ta': '기온(°C)',
    'ws': '풍속(m/s)',
    'wd': '풍향(16방위)',
    'hm': '습도(%)',
    'dc10Tca': '전운량(10분위)',
    'rn': '강수량(mm)',        # 추가
    'pv': '증기압(hPa)',
    'ps': '해면기압(hPa)',
    'ss': '일조(hr)',
    'dc10LmcsCa': '중하층운량(10분위)'
}

weather = df[col_need].copy()
weather = weather.rename(columns=dict_col)
weather

,일시,지점,지점명,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),강수량(mm),증기압(hPa),해면기압(hPa),일조(hr),중하층운량(10분위)
0,2015-01-01 00:00,152,울산,None,6.5,290,None,None,None,None,None,None,None
1,2015-01-01 01:00,152,울산,-1.0,4.5,270,42,None,None,2.4,1017.4,None,None
2,2015-01-01 02:00,152,울산,-1.3,3.4,320,39,None,None,2.2,1017.6,None,None
3,2015-01-01 03:00,152,울산,-2.5,5.0,320,49,4,None,2.5,1018.6,None,4
4,2015-01-01 04:00,152,울산,-3.3,6.2,290,36,0,None,1.7,1018.3,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
716,2015-01-30 20:00,152,울산,1.6,4.0,320,28,None,None,1.9,1023.6,None,None
717,2015-01-30 21:00,152,울산,1.0,3.4,320,28,0,None,1.8,1024.5,None,0
718,2015-01-30 22:00,152,울산,0.3,3.2,340,31,None,None,1.9,1025.0,None,None
719,2015-01-30 23:00,152,울산,-0.1,3.9,320,32,None,None,1.9,1025.5,None,None


# -------------------------

In [97]:
obs_dangjin = pd.DataFrame()
tmp1 = get_obs_dangjin('20180301', '20180331', ServiceKey)
tmp2 = get_obs_dangjin('20180401', '20180430', ServiceKey)

obs_dangjin = pd.concat([obs_ulsan, tmp1])
obs_dangjin = pd.concat([obs_ulsan, tmp2])
obs_dangjin

Success!
Success!


,일시,지점,지점명,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),강수량(mm),증기압(hPa),해면기압(hPa),일조(hr),중하층운량(10분위)
0,2018-03-01 00:00,152,울산,8.2,3.9,340,98,10,1.2,10.6,992.1,None,8
1,2018-03-01 01:00,152,울산,7.0,4.1,320,97,10,None,9.7,993.9,None,5
2,2018-03-01 02:00,152,울산,6.5,5.9,290,80,None,None,7.7,996.6,None,1
3,2018-03-01 03:00,152,울산,6.2,4.6,320,79,3,0.2,7.5,997.9,None,3
4,2018-03-01 04:00,152,울산,6.7,4.5,320,73,1,None,7.1,999.1,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,2018-04-30 19:00,129,서산,18.6,1.4,270,86,None,None,18.4,1015.0,0.1,None
716,2018-04-30 20:00,129,서산,17.8,1.4,230,91,None,None,18.5,1015.4,0.0,None
717,2018-04-30 21:00,129,서산,18.1,1.1,180,91,None,None,18.9,1016.4,None,None
718,2018-04-30 22:00,129,서산,18.0,0.6,200,91,None,None,18.8,1016.7,None,None


In [87]:
tmp

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'},
  'body': {'dataType': 'XML',
   'items': {'item': [{'tm': '2018-03-01 00:00',
      'rnum': '1',
      'stnId': '152',
      'stnNm': '울산',
      'ta': '8.2',
      'taQcflg': '0',
      'rn': '1.2',
      'rnQcflg': '0',
      'ws': '3.9',
      'wsQcflg': '0',
      'wd': '340',
      'wdQcflg': '0',
      'hm': '98',
      'hmQcflg': '0',
      'pv': '10.6',
      'td': '7.8',
      'pa': '982.2',
      'paQcflg': '0',
      'ps': '992.1',
      'psQcflg': '0',
      'ss': None,
      'ssQcflg': '9',
      'icsr': None,
      'dsnw': None,
      'hr3Fhsc': None,
      'dc10Tca': '10',
      'dc10LmcsCa': '8',
      'clfmAbbrCd': 'StNs',
      'lcsCh': None,
      'vs': '1357',
      'gndSttCd': None,
      'dmstMtphNo': '01',
      'ts': '7.6',
      'tsQcflg': '0',
      'm005Te': None,
      'm01Te': None,
      'm02Te': None,
      'm03Te': None},
     {'tm': '2018-03-01 01:00',
      'rnum': '2',
     

In [122]:
stnlds_ulsan = 152    # 울산
stnlds_dangjin = 129  # 서산(당진 옆)

url_base = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'
key = ServiceKey
queryParams = urlencode({
    quote_plus('pageNo'): 1,
    quote_plus('numOfRows'): 28 * 24,
    quote_plus('dataCd'): 'ASOS',
    quote_plus('dateCd'): 'HR',
    quote_plus('stnIds'): stnlds_ulsan,
    quote_plus('startDt'): '20200201', # 예보 발표 날짜
    quote_plus('startHh'): '00',       # 예보 발표 시간
    quote_plus('endDt'): '20200228',
    quote_plus('endHh'): '23',
})

url = f'{url_base}?serviceKey={key}&{queryParams}'
response = urlopen(url)
results = response.read().decode('utf-8')
results_json = xmltodict.parse(results)

data = json.loads(json.dumps(results_json))

if 'ERROR' in data['response']['header']['resultMsg']:
    print('Fail.')
else:
    print('Success!')
    




Success!


In [123]:
# data 대강 확인하기
data

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'},
  'body': {'dataType': 'XML',
   'items': {'item': [{'tm': '2020-02-01 00:00',
      'rnum': '1',
      'stnId': '152',
      'stnNm': '울산',
      'ta': '1.5',
      'taQcflg': None,
      'rn': None,
      'rnQcflg': None,
      'ws': '0.6',
      'wsQcflg': None,
      'wd': '320',
      'wdQcflg': None,
      'hm': '73',
      'hmQcflg': None,
      'pv': '5.0',
      'td': '-2.8',
      'pa': '1011.0',
      'paQcflg': None,
      'ps': '1021.5',
      'psQcflg': None,
      'ss': None,
      'ssQcflg': '9',
      'icsr': None,
      'dsnw': None,
      'hr3Fhsc': None,
      'dc10Tca': '0',
      'dc10LmcsCa': '0',
      'clfmAbbrCd': None,
      'lcsCh': None,
      'vs': '1256',
      'gndSttCd': None,
      'dmstMtphNo': '40',
      'ts': '0.1',
      'tsQcflg': None,
      'm005Te': None,
      'm01Te': None,
      'm02Te': None,
      'm03Te': None},
     {'tm': '2020-02-01 01:00',
      'rnum': '2',

In [52]:
logs = data['response']['body']['items']['item']
df = pd.DataFrame(logs)
df

,tm,rnum,stnId,stnNm,ta,taQcflg,rn,rnQcflg,ws,wsQcflg,...,lcsCh,vs,gndSttCd,dmstMtphNo,ts,tsQcflg,m005Te,m01Te,m02Te,m03Te
0,2020-02-01 00:00,1,152,울산,1.5,None,None,None,0.6,None,...,None,1256,None,40,0.1,None,None,None,None,None
1,2020-02-01 01:00,2,152,울산,2.5,None,None,None,2.1,None,...,None,1051,None,None,0.2,None,None,None,None,None
2,2020-02-01 02:00,3,152,울산,2.3,None,None,None,3.0,None,...,None,1187,None,None,0.3,None,None,None,None,None
3,2020-02-01 03:00,4,152,울산,1.6,None,None,None,3.2,None,...,None,945,None,40,0.2,None,None,None,None,None
4,2020-02-01 04:00,5,152,울산,1.7,None,None,None,2.8,None,...,None,859,None,40,0.2,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,2020-02-28 19:00,668,152,울산,5.8,None,None,9,0.4,None,...,6,1300,None,01,6.2,None,None,None,None,None
668,2020-02-28 20:00,669,152,울산,5.7,None,None,9,1.5,None,...,6,1500,None,None,5.9,None,None,None,None,None
669,2020-02-28 21:00,670,152,울산,5.5,None,0.1,None,1.7,None,...,6,1800,None,None,5.5,None,None,None,None,None
670,2020-02-28 22:00,671,152,울산,5.1,None,None,9,1.9,None,...,5,1800,None,None,4.5,None,None,None,None,None


In [75]:
df.columns

Index(['tm', 'rnum', 'stnId', 'stnNm', 'ta', 'taQcflg', 'rn', 'rnQcflg', 'ws',
       'wsQcflg', 'wd', 'wdQcflg', 'hm', 'hmQcflg', 'pv', 'td', 'pa',
       'paQcflg', 'ps', 'psQcflg', 'ss', 'ssQcflg', 'icsr', 'dsnw', 'hr3Fhsc',
       'dc10Tca', 'dc10LmcsCa', 'clfmAbbrCd', 'lcsCh', 'vs', 'gndSttCd',
       'dmstMtphNo', 'ts', 'tsQcflg', 'm005Te', 'm01Te', 'm02Te', 'm03Te'],
      dtype='object')

In [76]:
col_need_origin = ['tm', 'stnId', 'stnNm', 'ta', 'ws', 'wd', 'hm', 'dc10Tca']
col_need = [
    'tm', 'stnId', 'stnNm', 'ta', 'ws', 'wd', 'hm', 'dc10Tca',
    'rn', 'pv', 'ps', 'ss', 'icsr', 'dc10LmcsCa',
]
dict_col = {
    'stnId': '지점',
    'stnNm': '지점명',
    'tm': '일시',
    'ta': '기온(°C)',
    'ws': '풍속(m/s)',
    'wd': '풍향(16방위)',
    'hm': '습도(%)',
    'dc10Tca': '전운량(10분위)',
    'rn': '강수량(mm)',        # 추가
    'pv': '증기압(hPa)',
    'ps': '해면기압(hPa)',
    'ss': '일조(hr)',
    'icsr': '일사(MJ/m2)',
    'dc10LmcsCa': '중하층운량(10분위)'
}

weather = df[col_need].copy()
weather = weather.rename(columns=dict_col)

In [77]:
weather

,일시,지점,지점명,기온(°C),풍속(m/s),풍향(16방위),습도(%),전운량(10분위),강수량(mm),증기압(hPa),해면기압(hPa),일조(hr),일사(MJ/m2),중하층운량(10분위)
0,2020-02-01 00:00,152,울산,1.5,0.6,320,73,0,None,5.0,1021.5,None,None,0
1,2020-02-01 01:00,152,울산,2.5,2.1,340,69,0,None,5.0,1021.1,None,None,0
2,2020-02-01 02:00,152,울산,2.3,3.0,340,70,0,None,5.0,1021.5,None,None,0
3,2020-02-01 03:00,152,울산,1.6,3.2,340,75,0,None,5.2,1022.0,None,None,0
4,2020-02-01 04:00,152,울산,1.7,2.8,340,75,0,None,5.2,1022.1,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,2020-02-28 19:00,152,울산,5.8,0.4,0,98,10,None,9.0,1021.5,0.0,None,8
668,2020-02-28 20:00,152,울산,5.7,1.5,270,98,10,None,9.0,1021.1,None,None,10
669,2020-02-28 21:00,152,울산,5.5,1.7,290,98,6,0.1,8.8,1021.3,None,None,6
670,2020-02-28 22:00,152,울산,5.1,1.9,290,98,3,None,8.6,1021.0,None,None,3


In [71]:
weather['일조(hr)'].value_counts(dropna=False)

NaN    342
1.0    175
0.0     79
0.4     15
0.2     12
0.5     10
0.1      9
0.8      9
0.3      8
0.6      7
0.9      4
0.7      2
Name: 일조(hr), dtype: int64

In [72]:
weather['일사(MJ/m2)'].value_counts(dropna=False)

NaN    672
Name: 일사(MJ/m2), dtype: int64

In [78]:
weather['강수량(mm)'].value_counts(dropna=False)

NaN    644
0.0      9
2.2      2
0.2      2
0.6      2
0.1      1
6.0      1
0.7      1
6.2      1
2.8      1
3.0      1
6.5      1
3.2      1
1.0      1
1.1      1
5.0      1
7.9      1
4.1      1
Name: 강수량(mm), dtype: int64

In [79]:
weather['중하층운량(10분위)'].value_counts(dropna=False)

0     377
8      46
9      41
7      35
1      34
6      34
3      30
4      26
2      23
5      20
10      6
Name: 중하층운량(10분위), dtype: int64